In [1]:
import torch
import pandas as pd
import json
from Bio import SeqIO
from tqdm.auto import tqdm

train_data = torch.load('../data/ec/sprot_10_1022_esm2_t33_ec_above_10_single_label_train.pt')
val_data = torch.load('../data/ec/sprot_10_1022_esm2_t33_ec_above_10_single_label_val.pt')
test_data = torch.load('../data/ec/sprot_10_1022_esm2_t33_ec_above_10_single_label_test.pt')
print(f'train: {len(train_data)}, val: {len(val_data)}, test: {len(test_data)}')
# train_data.update(val_data)
# train_data.update(test_data)
data = train_data
print(f'all data: {len(data)}')
records = list(SeqIO.parse('../data/swissprot/uniprot_sprot_10_1022.fasta', 'fasta'))
pid2seq = {r.id.split('|')[1]: str(r.seq) for r in records}
pid_ec_seq = []
for pid in tqdm(data):
    ec = data[pid]['ec'][0]
    seq = pid2seq[pid]
    pid_ec_seq.append((pid, ec, seq))
print(f'pid_ec_seq: {len(pid_ec_seq)}')
df = pd.DataFrame(pid_ec_seq, columns=['Entry', 'EC number', 'Sequence'])
df.to_csv('/work/jiaqi/CLEAN/app/data/sprot_10_1022_ec_above_10_single_label_train.csv', index=False, sep='\t')

train: 167108, val: 20888, test: 20889
all data: 167108


  0%|          | 0/167108 [00:00<?, ?it/s]

pid_ec_seq: 167108


In [2]:
with open('/work/jiaqi/CLEAN/app/data/sprot_10_1022_ec_above_10_single_label_train.fasta', 'w') as f:
    for pid, _, seq in pid_ec_seq:
        f.write(f'>{pid}\n{seq}\n')

In [3]:
import os

cached_list = []
for root, dirs, files in os.walk('/work/jiaqi/CLEAN/app/data/esm_data'):
    for file in files:
        cached_list.append(file.split('.')[0])
cached_list = set(cached_list)
all_pid = set(data.keys())
print(f'cached: {len(cached_list)}, all: {len(all_pid)}')
to_cache = all_pid - cached_list
print(f'to cache: {len(to_cache)}')

cached: 209225, all: 208885
to cache: 0


In [1]:
import torch

CLEAN_emb = torch.load('../data/ec/sprot_10_1022_CLEAN_ec_above_10_single_label.pt')
train_esm = torch.load('../data/ec/sprot_10_1022_esm2_t33_ec_above_10_single_label_train.pt')
test_esm = torch.load('../data/ec/sprot_10_1022_esm2_t33_ec_above_10_single_label_test.pt')
val_esm = torch.load('../data/ec/sprot_10_1022_esm2_t33_ec_above_10_single_label_val.pt')
train_pids = list(train_esm.keys())
val_pids = list(val_esm.keys())
test_pids = list(test_esm.keys())
print(f'train: {len(train_pids)}, val: {len(val_pids)}, test: {len(test_pids)}')
train_clean_emb = {pid: {'embedding': CLEAN_emb[pid], 'ec': train_esm[pid]['ec']} for pid in train_pids}
val_clean_emb = {pid: {'embedding': CLEAN_emb[pid], 'ec': val_esm[pid]['ec']} for pid in val_pids}
test_clean_emb = {pid: {'embedding': CLEAN_emb[pid], 'ec': test_esm[pid]['ec']} for pid in test_pids}
print(f'train: {len(train_clean_emb)}, val: {len(val_clean_emb)}, test: {len(test_clean_emb)}')
torch.save(train_clean_emb, '../data/ec/sprot_10_1022_CLEAN_ec_above_10_single_label_train.pt')
torch.save(val_clean_emb, '../data/ec/sprot_10_1022_CLEAN_ec_above_10_single_label_val.pt')
torch.save(test_clean_emb, '../data/ec/sprot_10_1022_CLEAN_ec_above_10_single_label_test.pt')

train: 167108, val: 20888, test: 20889
train: 167108, val: 20888, test: 20889
